In [3]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, binary_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer, RandomRotation, RandomFlip,Resizing, Rescaling
import tensorflow_probability as tfp
import cv2 as cv2
import albumentations as A

In [46]:
dataset, dataset_info = tfds.load('malaria', with_info = True, as_supervised=True, shuffle_files = True, split = ['train'])

def splits(dataset, TRAIN_RATIO , VAL_RATIO , TEST_RATIO, print_results = False):    
    
    DATASET_SIZE = len(dataset)
    train_dataset = dataset.take(int( TRAIN_RATIO * DATASET_SIZE))
    
    
    val_test_dataset = dataset.skip(int( TRAIN_RATIO * DATASET_SIZE))
    val_dataset = val_test_dataset.take(int( VAL_RATIO * DATASET_SIZE))
    
    
    test_dataset = val_test_dataset.skip(int( VAL_RATIO * DATASET_SIZE))
    

    if print_results == True:
        print(list(dataset.as_numpy_iterator()))
        print(list(train_dataset.as_numpy_iterator()))
        print(list(val_dataset.as_numpy_iterator()))
        print(list(test_dataset.as_numpy_iterator()))
    return train_dataset, val_dataset, test_dataset

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

# dataset = tf.data.Dataset.range(10)

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO , VAL_RATIO , TEST_RATIO) #here
print(list(train_dataset.take(1).as_numpy_iterator()), 
      list(val_dataset.take(1).as_numpy_iterator()), 
      list(test_dataset.take(1).as_numpy_iterator()))

IM_SIZE = 224
def resize_rescale(image, label):
    return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
# # test_dataset = test_dataset.map(resize_rescale)

[(array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), 1)] [(array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
     

In [47]:
model= tf.keras.Sequential([
    InputLayer (input_shape= (IM_SIZE, IM_SIZE, 3)),
    Conv2D(filters = 6, kernel_size = 5, strides=1, padding='valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),
    Conv2D(filters = 16, kernel_size = 5, strides=1, padding='valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),
    Flatten (),

    Dense (100, activation = "relu"),
    BatchNormalization(),
    Dense (10, activation = "relu"),
    BatchNormalization(),
    Dense (1, activation = "sigmoid"),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_4 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_5 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                      

# Custom Loss

In [48]:
class CustomBCE(tf.keras.losses.Loss):
    def __init__(self, FACTOR):
        super(CustomBCE, self).__init__()
        self.FACTOR = FACTOR

    def call(self,y_true, y_pred):
        bce = BinaryCrossentropy()
        return bce(y_true, y_pred) * self.FACTOR

In [49]:
FACTOR = 1
def custom_bce(FACTOR):
    def loss(self, y_true, y_pred):
        bce = BinaryCrossentropy()
        return bce(y_true, y_pred) * FACTOR
    return loss

# Custom Metrics

In [50]:
FACTOR = 1
# def custom_accuracy(y_true, y_pred):
#     return binary_accuracy(y_true, y_pred)

In [51]:
def custom_accuracy(FACTOR):
    def acc(y_true, y_pred):
        return binary_accuracy(tf.cast(y_true, dtype = tf.float32), y_pred) * FACTOR
    return acc

In [64]:
class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, FACTOR, name = "Custom_Accuracy"):
        super(CustomAccuracy, self).__init__()
        self.FACTOR = FACTOR
        self.accuracy = self.add_weight(name= name, initializer='zeros')

    def update_state(self,y_true, y_pred, sample_weight = None):
        output = binary_accuracy(tf.cast(y_true, dtype = tf.float32), y_pred) * self.FACTOR
        self.accuracy.assign(tf.math.count_nonzero(output, dtype = tf.float32)/ tf.cast(len(output), dtype = tf.float32))
        
    def result(self):
        return self.accuracy
        
    def reset_state(self):
        self.accuracy.assign(0.)

In [65]:
model.compile(optimizer=Adam(learning_rate = 0.01),
             loss = CustomBCE(FACTOR),
             metrics = CustomAccuracy(FACTOR)
             )

In [ ]:
with tf.device('/gpu:0'):
    history = model.fit(train_dataset, validation_data = val_dataset, epochs =3, verbose = 1)

Epoch 1/3
689/689 [==============================] - 16s 21ms/step - loss: 0.1902 - custom_accuracy_10: 0.9667 - val_loss: 0.1527 - val_custom_accuracy_10: 0.6667
Epoch 2/3
687/689 [============================>.] - ETA: 0s - loss: 0.1535 - custom_accuracy_10: 0.9375

2024-03-08 02:30:14.552393: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:349: Filling up shuffle buffer (this may take a while): 1 of 8
2024-03-08 02:30:14.770797: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
